In [2]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

In [3]:
# Obter dados socioeconômicos e notas do ENEM 2023
df_enem = pd.read_pickle('Bases\Finais\dados_escolares_2023.pkl')

# Remover linhas com valores não explicativos
df_enem = df_enem[
    (df_enem['CAT_ENSINO'] != 0) &
    (df_enem['CAT_ESCOLA'] != 1)
]
#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
# Apenas utilizado nos modelos fianais, BayesSearchCV não utiliza Eval Set já possui validacao cruzada interna
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

Modelo Base

In [4]:
# Treinar modelo LGBMRegressor Base
modelo_lgbm = lgbm.LGBMRegressor(n_estimators=1000, 
                                 learning_rate=0.01, 
                                 random_state=42,
                                 max_bin=4095,
                                 force_row_wise=True)

start_time = time.time()

modelo_lgbm.fit(X_train_final, 
                y_train_final['NUM_NOTA_CH'], 
                eval_set=[(X_test, y_test['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features)

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 30394
[LightGBM] [Info] Number of data points in the train set: 478696, number of used features: 73
[LightGBM] [Info] Start training from score 528.797023


In [5]:
# Previsões
y_pred = modelo_lgbm.predict(X_test)

In [6]:
nome_experimento = 'Escolares CH 2023'

registrar_modelo(experimento=nome_experimento,
                 parametros={**modelo_lgbm.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                 X_train=X_train,
                 y_train=y_train,
                 y_test=y_test,
                 y_pred=y_pred,
                 variavel_alvo='NUM_NOTA_CH',
                 modelo=modelo_lgbm,
                 nome_modelo='modelo_lgbm_base_escolares_2023',
                 descricao_modelo='Modelo LGBMRegressor base para dados escolares CH',)

2025/06/11 13:17:24 INFO mlflow.tracking.fluent: Experiment with name 'Escolares CH 2023' does not exist. Creating a new experiment.
2025/06/11 13:17:44 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
Successfully registered model 'modelo_lgbm_base_escolares_2023'.
2025/06/11 13:17:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_lgbm_base_escolares_2023, version 1


🏃 View run overjoyed-turtle-87 at: http://127.0.0.1:9080/#/experiments/623589849857672017/runs/89bcdfd7e5e04d0f92d535887452118b
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/623589849857672017
Modelo registrado com sucesso no MLflow: modelo_lgbm_base_escolares_2023
Rastreamento do MLflow finalizado.


Created version '1' of model 'modelo_lgbm_base_escolares_2023'.


In [7]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred, "teste")

MAE (treino): 58.3056
RMSE (treino): 73.4808
R2 (treino): 0.2425
MAE (teste): 58.8803
RMSE (teste): 74.1800
R2 (teste): 0.2282


Bayes Search

In [8]:
modelo_lgbm_bayes = lgbm.LGBMRegressor(random_state=42,
                                       max_bin=4095, 
                                       force_row_wise=True)

In [9]:
# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (5, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (60, 120),                        # Profundidade máxima da árvore
    'learning_rate': (0.001, 0.01, 'log-uniform'), # Taxa de aprendizado
    'n_estimators': (5000, 8000),                  # Número de árvores
    'subsample': (0.1, 0.9),                       # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.1, 0.9),                # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1.0, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-7, 1e-2, 'log-uniform'),     # Regularização L2
}

In [10]:
# Configurar a busca Bayesiana usando BayesSearchCV

# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_lgbm_bayes,    # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=5,                       # Número de avaliações do modelo
    cv=5,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

In [11]:
fit_params = {
    'eval_metric': ['r2', 'rmse', 'mae'],              # Métricas a serem avaliadas
    'categorical_feature': categorical_features,       # Colunas categóricas
}

In [12]:
# Executar a busca Bayesiana

start_time = time.time()
bayes_search.fit(X_train, y_train['NUM_NOTA_CH'], **fit_params)

# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Info] Total Bins 30475
[LightGBM] [Info] Number of data points in the train set: 563172, number of used features: 73
[LightGBM] [Info] Start training from score 528.818375


In [ ]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = bayes_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
    print("R2: ", bayes_search.best_score_)
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")
except:
    melhores_parametros = {'colsample_bytree': 0.4558660098409215, 'learning_rate': 0.008293207307063736, 'max_depth': 66, 'n_estimators': 6300, 'num_leaves': 15, 'reg_alpha': 0.02296127666011783, 'reg_lambda': 5.987427535052461e-07, 'subsample': 0.7012420250723972}
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Melhores parâmetros: OrderedDict([('colsample_bytree', 0.4558660098409215), ('learning_rate', 0.008293207307063736), ('max_depth', 66), ('n_estimators', 6300), ('num_leaves', 15), ('reg_alpha', 0.02296127666011783), ('reg_lambda', 5.987427535052461e-07), ('subsample', 0.7012420250723972)])
R2:  0.22851773701869255
Tempo total de execução: 23330.66 segundos


In [14]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.fit(X_train_final, 
                      y_train_final['NUM_NOTA_CH'], 
                      eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                      eval_metric=['r2', 'rmse', 'mae'],
                      categorical_feature=categorical_features,
                      callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 30394
[LightGBM] [Info] Number of data points in the train set: 478696, number of used features: 73
[LightGBM] [Info] Start training from score 528.797023
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[4054]	valid_0's rmse: 74.1533	valid_0's l1: 58.8522	valid_0's l2: 5498.71


In [15]:
# Previsões
y_pred_bayes = modelo_lgbm_bayes.predict(X_test)

In [16]:
nome_experimento = 'Escolares CH 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_bayes,
                    parametros={**modelo_lgbm_bayes.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_bayes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_bayes_escolares_2023',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV para dados escolares CH',)

2025/06/11 19:55:37 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
Successfully registered model 'modelo_lgbm_bayes_escolares_2023'.
2025/06/11 19:55:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_lgbm_bayes_escolares_2023, version 1


🏃 View run invincible-hound-299 at: http://127.0.0.1:9080/#/experiments/623589849857672017/runs/4698a5dbdcff4d00aab1c92c993655dc
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/623589849857672017
Modelo registrado com sucesso no MLflow: modelo_lgbm_bayes_escolares_2023
Rastreamento do MLflow finalizado.


Created version '1' of model 'modelo_lgbm_bayes_escolares_2023'.


In [17]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_bayes.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_bayes, "teste")

MAE (treino): 58.3570
RMSE (treino): 73.5440
R2 (treino): 0.2412
MAE (teste): 58.8581
RMSE (teste): 74.1456
R2 (teste): 0.2290


In [18]:
# Salvar modelo como Pickle
joblib.dump(modelo_lgbm_bayes, 'modelos/modelo_lgbm_bayes_escolares.pkl')

['modelos/modelo_lgbm_bayes_escolares.pkl']